In [10]:
#%%writefile tools.py
import nacl.public, nacl.encoding
import os
import stat
import gzip

def create_keys(fullname):
    ''' Create secret and public keys for nacl
    
        The keys are stored in path in hexencoding where
        path,name = os.split(fullname)
        
        If path does not exist, raise an error
        If path has wrong permissions, raise an error:
        Only onwner can read,write or execute path (i.e. chmod path og-rwx)
        
        The permissions of the resulting secret keyfile is og-rwx
        (Only owner can read,write or execute secret keyfile)
    '''
    path,name = os.path.split(fullname)
    pub_name = fullname+'.pub'
    
    assert os.path.exists(path), f"path '{path}' does not exist"
    
    # user, group and others permissions on path must be 0o700
    # what means: user can read (1), write (2), execute (4) , 1+2+4 = 0o7
    #             group can not read, write or execute (0)            0o0
    #             others can not read, write or execute (0)           0o0
    assert (os.stat(path).st_mode & 0o777) == 0o700, \
    f"Mode of path: '{path}' must be 'rwx------' "\
    f"but is '{stat.filemode(os.stat(path).st_mode)[1:]}'"
    
    assert not os.path.exists(fullname), f"keyfile: '{fullname}' exists already"
    
    with open(fullname,'w') as s_f, open(pub_name,'w') as p_f:
        key = nacl.public.PrivateKey.generate()
        
        s_f.write(key.encode(nacl.encoding.HexEncoder).decode())
        p_f.write(key.public_key.encode(nacl.encoding.HexEncoder).decode())
        
    os.chmod(fullname,0o700)

def read_secret_key(fullname):
    ''' Read a private key for nacl from fullname.
        The path and the fullname must have the right permissions, og-rwx
        (Only user can read,write or execute secret key)
    '''
    path,name = os.path.split(fullname)
    assert (os.stat(path).st_mode & 0o777) == 0o700, \
    f"Mode of path '{path}' must be 'rwx------' "\
    f"but is '{stat.filemode(os.stat(path).st_mode)[1:]}'"
    assert (os.stat(fullname).st_mode & 0o777) == 0o700, \
    f"Mode of file '{fullname}' must be 'rwx------' "\
    f"but is {stat.filemode(os.stat(fullname).st_mode)[1:]}"
    
    with open(fullname,'r') as f:
        key = nacl.public.PrivateKey(f.read().encode(),encoder=nacl.encoding.HexEncoder)
        
    return key

def read_public_key(fullname):
    ''' Read a public key for nacl from fullname'''
    
    with open(fullname,'r') as f:
        key = nacl.public.PublicKey(f.read().encode(),encoder=nacl.encoding.HexEncoder)
    
    return key

def encrypt(sk,pk,f_in_name,f_out_name):
    ''' encrypt f_in_name to f_out_name using nacl with 
        secret key sk and public key pk. If f_out_name
        ends with '.gz', use gzip
    '''
    gzipped = os.path.splitext(f_out_name)[1] == '.gz'
    out = gzip.open if gzipped else open
    
    with open(f_in_name,'rb') as f_in, out(f_out_name,'wb') as f_out:
        f_out.write(nacl.public.Box(sk,pk).encrypt(f_in.read()))


def decrypt(sk,pk,f_in_name,f_out_name):
    ''' decrypt f_in_name to f_out_name using nacl with
        secret key sk und public key pk. If f_in_name ends
        with '.gz' use gzip
    '''
    gzipped = os.path.splitext(f_in_name)[1] == '.gz'
    in_ = gzip.open if gzipped else open
    with in_(f_in_name,'rb') as f_in, open(f_out_name,'wb') as f_out:
        f_out.write(nacl.public.Box(sk,pk).decrypt(f_in.read()))


Overwriting tools.py


In [ ]:
from tools import *

In [2]:
!rm -rf ./.encr/

In [3]:
!ls -hal

total 96
drwxr-xr-x  14 wolfgang  wheel   448B  9 Jul 21:28 .
drwxr-xr-x   3 wolfgang  wheel    96B  9 Jul 18:54 ..
drwxr-xr-x  12 wolfgang  wheel   384B  9 Jul 18:55 .git
-rw-r--r--   1 wolfgang  wheel    37B  9 Jul 18:54 .gitignore
drwxr-xr-x   3 wolfgang  wheel    96B  9 Jul 18:58 .ipynb_checkpoints
-rw-r--r--   1 wolfgang  wheel   6,5K  9 Jul 18:54 Datei_verschluesseln_und_entschluesseln.ipynb
-rw-r--r--   1 wolfgang  wheel   600B  9 Jul 18:54 README.md
-rw-r--r--   1 wolfgang  wheel    18K  9 Jul 21:28 Untitled.ipynb
drwxr-xr-x   3 wolfgang  wheel    96B  9 Jul 21:13 __pycache__
drwxr-xr-x   5 wolfgang  wheel   160B  9 Jul 18:54 alice
drwxr-xr-x   5 wolfgang  wheel   160B  9 Jul 18:54 bob
-rw-r--r--   1 wolfgang  wheel    18B  9 Jul 18:54 text.txt
-rw-r--r--   1 wolfgang  wheel   2,9K  9 Jul 18:54 tools.ipynb
-rw-r--r--   1 wolfgang  wheel   3,4K  9 Jul 21:13 tools.py


In [4]:
create_keys('./.encr/secret_key')

AssertionError: path './.encr' does not exist

In [5]:
!mkdir ./.encr
create_keys('./.encr/secret_key')

AssertionError: Mode of path: './.encr' must be 'rwx------' but is 'rwxr-xr-x'

In [6]:
!chmod og-rwx ./.encr
create_keys('./.encr/secret_key')

In [7]:
create_keys('./.encr/secret_key')

AssertionError: keyfile: './.encr/secret_key' exists already

In [8]:
!ls -hal 

total 96
drwxr-xr-x  15 wolfgang  wheel   480B  9 Jul 21:28 .
drwxr-xr-x   3 wolfgang  wheel    96B  9 Jul 18:54 ..
drwx------   4 wolfgang  wheel   128B  9 Jul 21:28 .encr
drwxr-xr-x  12 wolfgang  wheel   384B  9 Jul 18:55 .git
-rw-r--r--   1 wolfgang  wheel    37B  9 Jul 18:54 .gitignore
drwxr-xr-x   3 wolfgang  wheel    96B  9 Jul 18:58 .ipynb_checkpoints
-rw-r--r--   1 wolfgang  wheel   6,5K  9 Jul 18:54 Datei_verschluesseln_und_entschluesseln.ipynb
-rw-r--r--   1 wolfgang  wheel   600B  9 Jul 18:54 README.md
-rw-r--r--   1 wolfgang  wheel    18K  9 Jul 21:28 Untitled.ipynb
drwxr-xr-x   3 wolfgang  wheel    96B  9 Jul 21:13 __pycache__
drwxr-xr-x   5 wolfgang  wheel   160B  9 Jul 18:54 alice
drwxr-xr-x   5 wolfgang  wheel   160B  9 Jul 18:54 bob
-rw-r--r--   1 wolfgang  wheel    18B  9 Jul 18:54 text.txt
-rw-r--r--   1 wolfgang  wheel   2,9K  9 Jul 18:54 tools.ipynb
-rw-r--r--   1 wolfgang  wheel   3,4K  9 Jul 21:13 tools.py


In [9]:
!ls -hal ./.encr/

total 16
drwx------   4 wolfgang  wheel   128B  9 Jul 21:28 .
drwxr-xr-x  15 wolfgang  wheel   480B  9 Jul 21:28 ..
-rwx------   1 wolfgang  wheel    64B  9 Jul 21:28 secret_key
-rw-r--r--   1 wolfgang  wheel    64B  9 Jul 21:28 secret_key.pub
